In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import litellm

response = litellm.completion(
    model="gpt-4o-mini-2024-07-18",
    messages=[{"role": "user", "content": "hi"}],
    num_retries=10,
)
response.choices[0].message.content

'Hello! How can I assist you today?'